In [5]:
import random
import numpy as np
from collections import defaultdict, deque
import matplotlib.pyplot as plt
import networkx as nx
import math

class GraphColoringProblem:
    def __init__(self, edges, preassigned_colors=None, distance_constraints=None, k_beam_size=10, max_iterations=100):
        """
        Initialize the graph coloring problem.
        
        Parameters:
        - edges: List of tuples representing edges (source, destination)
        - preassigned_colors: Dictionary mapping vertex indices to fixed colors
        - distance_constraints: List of vertex pairs with distance constraint
        - k_beam_size: Number of states to maintain in the beam
        - max_iterations: Maximum number of iterations to run
        """
        self.edges = edges
        self.preassigned_colors = preassigned_colors or {}
        self.distance_constraints = distance_constraints or []
        self.k_beam_size = k_beam_size
        self.max_iterations = max_iterations
        
        # Extract vertices and build graph structure
        self.vertices = self._get_vertices()
        self.adj_list = self._create_adj_list()
        self.vertex_degrees = self._calculate_degrees()
        self.two_hop_neighbors = self._calculate_two_hop_neighbors()
        
    def _get_vertices(self):
        """Extract unique vertices from edges"""
        vertices = set()
        for src, dst in self.edges:
            vertices.add(src)
            vertices.add(dst)
        return vertices
    
    def _create_adj_list(self):
        """Create adjacency list representation of the graph"""
        adj_list = defaultdict(set)
        for src, dst in self.edges:
            adj_list[src].add(dst)
            adj_list[dst].add(src)
        return adj_list
    
    def _calculate_degrees(self):
        """Calculate degree of each vertex"""
        return {v: len(self.adj_list[v]) for v in self.vertices}
    
    def _calculate_two_hop_neighbors(self):
        """Calculate two-hop neighbors for each vertex"""
        two_hop = defaultdict(set)
        for v in self.vertices:
            # Get direct neighbors
            neighbors = self.adj_list[v]
            # Get neighbors of neighbors
            for neighbor in neighbors:
                two_hop[v].update(self.adj_list[neighbor])
            # Remove the vertex itself and direct neighbors
            two_hop[v] = two_hop[v] - {v} - neighbors
        return two_hop
        
    def is_valid_coloring(self, coloring):
        """
        Check if the coloring is valid according to all constraints
        """
        # Check adjacent vertex constraint
        for v in self.vertices:
            if v in coloring:
                for neighbor in self.adj_list[v]:
                    if neighbor in coloring and coloring[v] == coloring[neighbor]:
                        return False
        
        # Check preassigned colors
        for v, color in self.preassigned_colors.items():
            if v in coloring and coloring[v] != color:
                return False
        
        # Check distance constraints (two-hop)
        for v1, v2 in self.distance_constraints:
            if v1 in coloring and v2 in coloring and coloring[v1] == coloring[v2]:
                return False
            
        return True
    
    def calculate_color_balance(self, coloring):
        """
        Calculate the balance score of color usage
        Returns a value between 0 and 1, where 1 is perfectly balanced
        """
        if not coloring:
            return 0
            
        color_counts = defaultdict(int)
        for color in coloring.values():
            color_counts[color] += 1
            
        min_count = min(color_counts.values())
        max_count = max(color_counts.values())
        
        if max_count == min_count:
            return 1.0
        return 1.0 - (max_count - min_count) / len(self.vertices)
    
    def generate_initial_state(self):
        """
        Generate a valid initial state with all vertices colored,
        respecting preassigned colors
        """
        # Start with preassigned colors
        coloring = self.preassigned_colors.copy()
        
        # Get vertices sorted by degree (highest to lowest)
        vertices = sorted(self.vertices, key=lambda x: self.vertex_degrees[x], reverse=True)
        
        # Create a mapping of constrained pairs for quick lookup
        constrained_pairs = defaultdict(set)
        for v1, v2 in self.distance_constraints:
            constrained_pairs[v1].add(v2)
            constrained_pairs[v2].add(v1)
        
        for vertex in vertices:
            if vertex not in coloring:
                # Get colors of adjacent vertices
                neighbor_colors = {coloring[n] for n in self.adj_list[vertex] if n in coloring}
                
                # Only check distance constraints if this vertex is in constrained pairs
                forbidden_colors = set(neighbor_colors)
                if vertex in constrained_pairs:
                    # Add colors of constrained vertices to forbidden colors
                    for constrained_vertex in constrained_pairs[vertex]:
                        if constrained_vertex in coloring:
                            forbidden_colors.add(coloring[constrained_vertex])
                
                # Find the first available color
                color = 0
                while color in forbidden_colors:
                    color += 1
                coloring[vertex] = color
                
        return coloring
    
    def generate_successor(self, current_state):
        """
        Generate a neighboring state by changing one vertex's color
        """
        neighbor = current_state.copy()
        
        # Sort vertices by degree for modification priority
        vertices = sorted(self.vertices, key=lambda x: self.vertex_degrees[x], reverse=True)
        
        # Randomly select a vertex that's not preassigned
        modifiable_vertices = [v for v in vertices if v not in self.preassigned_colors]
        
        if not modifiable_vertices:
            return current_state
            
        vertex = random.choice(modifiable_vertices)
        
        # Get current colors of neighbors
        neighbor_colors = {neighbor[n] for n in self.adj_list[vertex] if n in neighbor}
        
        # Get colors of vertices with distance constraints
        constrained_colors = set()
        for v1, v2 in self.distance_constraints:
            if v1 == vertex and v2 in neighbor:
                constrained_colors.add(neighbor[v2])
            elif v2 == vertex and v1 in neighbor:
                constrained_colors.add(neighbor[v1])
        
        # Get all colors currently in use
        current_colors = set(neighbor.values())
        
        # Find available colors (including potentially a new color)
        available_colors = [c for c in range(max(current_colors) + 2) 
                          if c not in neighbor_colors and c not in constrained_colors]
        
        # Choose a random available color that's different from current
        available_colors = [c for c in available_colors if c != neighbor[vertex]]
        
        if available_colors:
            neighbor[vertex] = random.choice(available_colors)
            
        return neighbor
    
    def evaluate_state(self, state):
        """
        Evaluate a state based on number of colors and balance
        Returns a score where higher is better
        """
        if not self.is_valid_coloring(state):
            return float('-inf')
            
        num_colors = len(set(state.values()))
        balance_score = self.calculate_color_balance(state)
        
        # Combine metrics (weighted sum)
        return -0.7 * num_colors + 0.3 * balance_score
    
    def local_beam_search(self):
        """
        Perform local beam search to find optimal coloring
        """
        # Generate k initial states
        states = [self.generate_initial_state() for _ in range(self.k_beam_size)]
        states = sorted(states, key=self.evaluate_state, reverse=True)
        
        best_state = states[0]
        best_score = self.evaluate_state(best_state)
        
        for iteration in range(self.max_iterations):
            # Generate successors for all current states
            all_successors = []
            for state in states:
                # Generate multiple successors for each state
                for _ in range(self.k_beam_size):
                    successor = self.generate_successor(state)
                    all_successors.append(successor)
            
            # Combine current states with successors
            all_states = states + all_successors
            
            # Remove duplicates (convert dict to tuple of items for hashability)
            unique_states = []
            seen = set()
            for state in all_states:
                state_tuple = tuple(sorted(state.items()))
                if state_tuple not in seen:
                    unique_states.append(state)
                    seen.add(state_tuple)
            
            # Sort states by score and select k best
            states = sorted(unique_states, key=self.evaluate_state, reverse=True)[:self.k_beam_size]
            
            # Update best state if improved
            current_best = states[0]
            current_score = self.evaluate_state(current_best)
            
            if current_score > best_score:
                best_state = current_best
                best_score = current_score
                print(f"Iteration {iteration}: New best score = {best_score}, Colors = {len(set(best_state.values()))}")
            
            # Early stopping if all states are the same
            if all(self.evaluate_state(s) == self.evaluate_state(states[0]) for s in states):
                print(f"Early stopping at iteration {iteration} - No improvement possible")
                break
        
        return best_state

def read_hypercube_dataset(filename):
    """
    Read the hypercube dataset from file
    """
    edges = []
    with open(filename, 'r') as f:
        next(f)  # Skip the header line
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 2:
                src = int(parts[0])
                dst = int(parts[1])
                edges.append((src, dst))
    return edges

def plot_colored_graph(edges, coloring, distance_constraints, filename="colored_graph.png"):
    """
    Plot the graph with assigned colors and save it to a file
    
    Args:
        edges: List of tuples representing edges (source, destination)
        coloring: Dictionary mapping vertices to their assigned colors
        distance_constraints: List of vertex pairs with distance constraints
        filename: Name of the output file
    """
    # Print execution information
    print(f"Graph Visualization Started")
    print(f"User: Mustafaiqbal2")  # Hardcoded username instead of __file__

    # Create a new graph
    G = nx.Graph()
    
    # Add edges to the graph
    G.add_edges_from(edges)
    
    # Create a color map for visualization
    color_list = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
                 '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
                 '#aec7e8', '#ffbb78', '#98df8a', '#ff9896', '#c5b0d5']
    
    # Map color numbers to actual colors
    node_colors = [color_list[coloring[node] % len(color_list)] for node in G.nodes()]
    
    # Set up the plot
    plt.figure(figsize=(20, 20))
    
    # Create a spring layout with more spread
    pos = nx.spring_layout(G, k=1/math.sqrt(len(G.nodes())), iterations=50)
    
    # Draw the graph elements
    # Draw edges
    nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.5)
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, 
                          node_color=node_colors,
                          node_size=500)
    
    # Draw labels
    nx.draw_networkx_labels(G, pos, font_size=8)
    
    # Draw distance constraint edges in red with dashed lines
    constraint_edges = [(v1, v2) for v1, v2 in distance_constraints]
    nx.draw_networkx_edges(G, pos,
                          edgelist=constraint_edges,
                          edge_color='red',
                          style='dashed',
                          alpha=0.7,
                          width=2)
    
    # Add a title with statistics
    num_colors = len(set(coloring.values()))
    plt.title(f'Graph Coloring Visualization\n'
              f'Number of Vertices: {len(G.nodes())}\n'
              f'Number of Edges: {len(G.edges())}\n'
              f'Number of Colors Used: {num_colors}\n'
              f'Distance Constraints: {len(distance_constraints)}', 
              pad=20)
    
    # Add legend for distance constraints
    constraint_line = plt.Line2D([], [], color='red', linestyle='--',
                               label='Distance Constraint')
    plt.legend([constraint_line], ['Distance Constraint'])
    
    # Adjust layout and save
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Graph visualization has been saved to {filename}")

def main():
    # Read the hypercube dataset
    edges = read_hypercube_dataset('hypercube_dataset.txt')
    
    # Define preassigned colors to vertices
    preassigned_colors = {
        0: 0,   
        512: 1,  
        256: 2,   
        1023: 3,
        1022: 2,
        511: 4,
    }
    
    # distance constraints: add 2 hop vertices that must have different colors
    distance_constraints = [
        (945, 951),   
        (995, 1019), 
        (913, 1009),
        (271, 415),
        (134, 647), 
        (1013, 1023),
        (779, 827)
    ]
    
    # Create graph coloring instance
    graph = GraphColoringProblem(
        edges, 
        preassigned_colors=preassigned_colors,
        distance_constraints=distance_constraints,
        k_beam_size=5,
        max_iterations=100
    )
    
    print("Starting local beam search...")
    solution = graph.local_beam_search()
    
    # Check if the solution is valid
    if not graph.is_valid_coloring(solution):
        print("Solution is invalid!")
    else:
        print("Solution is valid.")
    
    # Print results
    print("\nFinal coloring statistics:")
    print(f"Number of colors used: {len(set(solution.values()))}")
    print(f"Color balance score: {graph.calculate_color_balance(solution):.4f}")
    
    # Print color distribution
    color_counts = defaultdict(int)
    for color in solution.values():
        color_counts[color] += 1
    
    print("\nColor distribution:")
    for color, count in sorted(color_counts.items()):
        print(f"Color {color}: {count} vertices")
    
    # Visualize the result
    print("\nGenerating visualization...")
    plot_colored_graph(
        edges=edges,
        coloring=solution,
        distance_constraints=distance_constraints,
        filename="graph_coloring_result.png"
    )

if __name__ == "__main__":
    main()

Starting local beam search...
Iteration 0: New best score = -3.348828125, Colors = 5
Early stopping at iteration 0 - No improvement possible
Solution is valid.

Final coloring statistics:
Number of colors used: 5
Color balance score: 0.5039

Color distribution:
Color 0: 509 vertices
Color 1: 504 vertices
Color 2: 8 vertices
Color 3: 2 vertices
Color 4: 1 vertices

Generating visualization...
Graph Visualization Started
User: Mustafaiqbal2
Graph visualization has been saved to graph_coloring_result.png


In [3]:
import random
import numpy as np
import pandas as pd
from collections import defaultdict

class Product:
    def __init__(self, id, name, weight, category, perishable=False, high_demand=False, 
                 hazardous=False, promotional=False, high_value=False, bulky=False):
        self.id = id
        self.name = name
        self.weight = weight
        self.category = category
        self.perishable = perishable
        self.high_demand = high_demand
        self.hazardous = hazardous
        self.promotional = promotional
        self.high_value = high_value
        self.bulky = bulky
        
    def __repr__(self):
        return f"{self.name} ({self.weight}kg)"

class Shelf:
    def __init__(self, id, name, capacity, is_refrigerated=False, is_eye_level=False, 
                 is_lower=False, is_hazardous_zone=False, is_secure=False, is_high_visibility=False):
        self.id = id
        self.name = name
        self.capacity = capacity
        self.is_refrigerated = is_refrigerated
        self.is_eye_level = is_eye_level
        self.is_lower = is_lower
        self.is_hazardous_zone = is_hazardous_zone
        self.is_secure = is_secure
        self.is_high_visibility = is_high_visibility
        
    def __repr__(self):
        return f"{self.name} ({self.capacity}kg)"

class ShelfOptimizationGA:
    def __init__(self, products, shelves, population_size=100, max_generations=500, 
                 mutation_rate=0.1, crossover_rate=0.8):
        self.products = products
        self.shelves = shelves
        self.population_size = population_size
        self.max_generations = max_generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        
        # Dictionary to store complementary products
        self.complementary_pairs = self._define_complementary_pairs()
        
    def _define_complementary_pairs(self):
        """Define pairs of products that complement each other"""
        # This is a simple example; in reality, this would be more complex
        pairs = []
        
        # Find products by category
        pasta_products = [p for p in self.products if "Pasta" in p.name]
        pasta_sauce_products = [p for p in self.products if "Sauce" in p.name]
        
        # Create complementary pairs
        for pasta in pasta_products:
            for sauce in pasta_sauce_products:
                pairs.append((pasta.id, sauce.id))
                
        return pairs
    
    def generate_individual(self):
        """Generate a random shelf assignment"""
        assignment = {}
        for product in self.products:
            # Randomly assign each product to a shelf
            valid_shelves = []
            
            for shelf in self.shelves:
                # Check basic constraints
                if product.perishable and not shelf.is_refrigerated:
                    continue
                if product.hazardous and not shelf.is_hazardous_zone:
                    continue
                
                valid_shelves.append(shelf.id)
            
            if valid_shelves:
                assignment[product.id] = random.choice(valid_shelves)
            else:
                # If no valid shelf found, assign randomly (will be penalized in fitness function)
                assignment[product.id] = random.choice([s.id for s in self.shelves])
                
        return assignment
    
    def generate_initial_population(self):
        """Generate initial population of shelf assignments"""
        return [self.generate_individual() for _ in range(self.population_size)]
    
    def calculate_fitness(self, individual):
        """Calculate fitness of an individual (lower is better)"""
        # Start with perfect fitness (0 penalties)
        fitness = 0
        
        # Check shelf capacity constraint
        shelf_weights = defaultdict(float)
        for product_id, shelf_id in individual.items():
            product = next(p for p in self.products if p.id == product_id)
            shelf_weights[shelf_id] += product.weight
        
        for shelf_id, weight in shelf_weights.items():
            shelf = next(s for s in self.shelves if s.id == shelf_id)
            if weight > shelf.capacity:
                # Penalize exceeding shelf capacity
                fitness += 50 * (weight - shelf.capacity)
        
        # Check accessibility of high-demand products
        for product in self.products:
            if product.high_demand:
                shelf_id = individual[product.id]
                shelf = next(s for s in self.shelves if s.id == shelf_id)
                if not shelf.is_eye_level:
                    # Penalize high-demand products not on eye-level shelves
                    fitness += 20
        
        # Check perishable items
        for product in self.products:
            if product.perishable:
                shelf_id = individual[product.id]
                shelf = next(s for s in self.shelves if s.id == shelf_id)
                if not shelf.is_refrigerated:
                    # Heavily penalize perishable items not in refrigeration
                    fitness += 100
        
        # Check hazardous items
        for product in self.products:
            if product.hazardous:
                shelf_id = individual[product.id]
                shelf = next(s for s in self.shelves if s.id == shelf_id)
                if not shelf.is_hazardous_zone:
                    # Heavily penalize hazardous items not in hazardous zone
                    fitness += 100
        
        # Check product compatibility and cross-selling
        for product1_id, product2_id in self.complementary_pairs:
            shelf1_id = individual[product1_id]
            shelf2_id = individual[product2_id]
            if shelf1_id != shelf2_id:
                # Penalize complementary products not on the same shelf
                fitness += 10
        
        # Check restocking efficiency
        for product in self.products:
            if product.bulky:
                shelf_id = individual[product.id]
                shelf = next(s for s in self.shelves if s.id == shelf_id)
                if not shelf.is_lower:
                    # Penalize bulky items not on lower shelves
                    fitness += 15
        
        # Check refrigeration efficiency
        refrigerated_shelves = [s.id for s in self.shelves if s.is_refrigerated]
        perishable_products = [p for p in self.products if p.perishable]
        
        for shelf_id in refrigerated_shelves:
            # Count products on this shelf
            products_on_shelf = [p for p in perishable_products if individual[p.id] == shelf_id]
            non_perishable_on_shelf = [product_id for product_id, s_id in individual.items() 
                                       if s_id == shelf_id and not next(p for p in self.products if p.id == product_id).perishable]
            
            # Penalize refrigerated shelves not fully utilized by perishable items
            if non_perishable_on_shelf:
                fitness += 10 * len(non_perishable_on_shelf)
        
        # Check promotional items visibility
        for product in self.products:
            if product.promotional:
                shelf_id = individual[product.id]
                shelf = next(s for s in self.shelves if s.id == shelf_id)
                if not shelf.is_high_visibility:
                    # Penalize promotional items not in high-visibility areas
                    fitness += 25
        
        # Check theft prevention
        for product in self.products:
            if product.high_value:
                shelf_id = individual[product.id]
                shelf = next(s for s in self.shelves if s.id == shelf_id)
                if not shelf.is_secure:
                    # Penalize high-value items not in secure areas
                    fitness += 40
        
        return fitness
    
    def select_parents(self, population, fitnesses):
        """Select parents using tournament selection"""
        tournament_size = 3
        selected_parents = []
        
        for _ in range(2):  # Select 2 parents
            tournament_indices = random.sample(range(len(population)), tournament_size)
            tournament_fitness = [fitnesses[i] for i in tournament_indices]
            winner_idx = tournament_indices[tournament_fitness.index(min(tournament_fitness))]
            selected_parents.append(population[winner_idx])
            
        return selected_parents
    
    def crossover(self, parent1, parent2):
        """Perform crossover between two parents"""
        if random.random() > self.crossover_rate:
            return parent1.copy(), parent2.copy()
            
        # Single-point crossover
        crossover_point = random.randint(1, len(self.products) - 1)
        
        child1 = {}
        child2 = {}
        
        product_ids = list(parent1.keys())
        
        for i, product_id in enumerate(product_ids):
            if i < crossover_point:
                child1[product_id] = parent1[product_id]
                child2[product_id] = parent2[product_id]
            else:
                child1[product_id] = parent2[product_id]
                child2[product_id] = parent1[product_id]
                
        return child1, child2
    
    def mutate(self, individual):
        """Randomly mutate an individual"""
        mutated = individual.copy()
        
        for product_id in mutated:
            if random.random() < self.mutation_rate:
                # Find valid shelves for this product
                product = next(p for p in self.products if p.id == product_id)
                valid_shelves = []
                
                for shelf in self.shelves:
                    # Check basic constraints
                    if product.perishable and not shelf.is_refrigerated:
                        continue
                    if product.hazardous and not shelf.is_hazardous_zone:
                        continue
                    
                    valid_shelves.append(shelf.id)
                
                if valid_shelves:
                    mutated[product_id] = random.choice(valid_shelves)
                    
        return mutated
    
    def run(self):
        """Run the genetic algorithm"""
        population = self.generate_initial_population()
        
        best_fitness = float('inf')
        best_individual = None
        
        for generation in range(self.max_generations):
            # Calculate fitness for each individual
            fitnesses = [self.calculate_fitness(ind) for ind in population]
            
            # Find the best individual
            gen_best_idx = fitnesses.index(min(fitnesses))
            gen_best_fitness = fitnesses[gen_best_idx]
            gen_best_individual = population[gen_best_idx]
            
            if gen_best_fitness < best_fitness:
                best_fitness = gen_best_fitness
                best_individual = gen_best_individual.copy()
                print(f"Generation {generation}: New best fitness = {best_fitness}")
                
                # If we found a perfect solution, stop
                if best_fitness == 0:
                    break
            
            # Create a new population
            new_population = []
            
            # Keep the best individual (elitism)
            new_population.append(gen_best_individual)
            
            # Fill the rest of the population
            while len(new_population) < self.population_size:
                # Select parents
                parent1, parent2 = self.select_parents(population, fitnesses)
                
                # Perform crossover
                child1, child2 = self.crossover(parent1, parent2)
                
                # Perform mutation
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)
                
                # Add children to new population
                new_population.append(child1)
                if len(new_population) < self.population_size:
                    new_population.append(child2)
            
            population = new_population
            
            if generation % 50 == 0:
                print(f"Generation {generation}: Best fitness = {best_fitness}")
        
        return best_individual, best_fitness
    
    def save_results_to_excel(self, solution, filename="shelf_optimization_results.xlsx"):
        """Save the optimization results to an Excel file"""
        # Create a DataFrame with the results
        results = []
        
        for product_id, shelf_id in solution.items():
            product = next(p for p in self.products if p.id == product_id)
            shelf = next(s for s in self.shelves if s.id == shelf_id)
            
            results.append({
                "Product ID": product_id,
                "Product Name": product.name,
                "Weight (kg)": product.weight,
                "Category": product.category,
                "Perishable": "Yes" if product.perishable else "No",
                "High Demand": "Yes" if product.high_demand else "No",
                "Hazardous": "Yes" if product.hazardous else "No",
                "Promotional": "Yes" if product.promotional else "No",
                "High Value": "Yes" if product.high_value else "No",
                "Bulky": "Yes" if product.bulky else "No",
                "Assigned Shelf": shelf.name,
                "Shelf Capacity (kg)": shelf.capacity
            })
        
        df = pd.DataFrame(results)
        
        # Calculate shelf weight utilization
        shelf_utilization = defaultdict(float)
        for product_id, shelf_id in solution.items():
            product = next(p for p in self.products if p.id == product_id)
            shelf_utilization[shelf_id] += product.weight
        
        shelf_stats = []
        for shelf in self.shelves:
            shelf_stats.append({
                "Shelf ID": shelf.id,
                "Shelf Name": shelf.name,
                "Capacity (kg)": shelf.capacity,
                "Used Capacity (kg)": shelf_utilization[shelf.id],
                "Utilization (%)": (shelf_utilization[shelf.id] / shelf.capacity) * 100 if shelf.capacity > 0 else 0
            })
        
        df_shelves = pd.DataFrame(shelf_stats)
        
        # Save to Excel
        with pd.ExcelWriter(filename) as writer:
            df.to_excel(writer, sheet_name="Product Assignments", index=False)
            df_shelves.to_excel(writer, sheet_name="Shelf Utilization", index=False)
            
        print(f"Results saved to {filename}")

# Example usage
if __name__ == "__main__":
    # Define shelves
    shelves = [
        Shelf("S1", "Checkout Display", 8, is_high_visibility=True),
        Shelf("S2", "Lower Shelf", 25, is_lower=True),
        Shelf("S4", "Eye-Level Shelf", 15, is_eye_level=True, is_high_visibility=True),
        Shelf("S5", "General Aisle Shelf", 20),
        Shelf("R1", "Refrigerator Zone", 20, is_refrigerated=True),
        Shelf("H1", "Hazardous Item Zone", 10, is_hazardous_zone=True)
    ]
    
    # Define products
    products = [
        Product("P1", "Milk", 5, "Dairy", perishable=True, high_demand=True),
        Product("P2", "Rice Bag", 10, "Grains", bulky=True),
        Product("P3", "Frozen Nuggets", 5, "Frozen", perishable=True),
        Product("P4", "Cereal", 3, "Breakfast", high_demand=True),
        Product("P5", "Pasta", 2, "Grains"),
        Product("P6", "Pasta Sauce", 3, "Condiments"),
        Product("P7", "Detergent", 4, "Cleaning", hazardous=True),
        Product("P8", "Glass Cleaner", 5, "Cleaning", hazardous=True)
    ]
    
    # Initialize and run the GA
    optimizer = ShelfOptimizationGA(products, shelves, population_size=50, max_generations=200)
    solution, fitness = optimizer.run()
    
    print("\nOptimized Shelf Allocation:")
    for product_id, shelf_id in solution.items():
        product = next(p for p in products if p.id == product_id)
        shelf = next(s for s in shelves if s.id == shelf_id)
        print(f"{product.name} -> {shelf.name}")
    
    print(f"\nFinal Fitness Score: {fitness}")
    
    # Check shelf capacities
    shelf_weights = defaultdict(float)
    for product_id, shelf_id in solution.items():
        product = next(p for p in products if p.id == product_id)
        shelf_weights[shelf_id] += product.weight
    
    print("\nShelf Weight Usage:")
    for shelf in shelves:
        weight = shelf_weights[shelf.id]
        print(f"{shelf.name}: {weight}kg / {shelf.capacity}kg")
    
    # Save results to Excel (requires pandas)
    optimizer.save_results_to_excel(solution, "shelf_optimization_results.xlsx")

Generation 0: New best fitness = 45
Generation 0: Best fitness = 45
Generation 1: New best fitness = 40
Generation 2: New best fitness = 30
Generation 4: New best fitness = 20
Generation 50: Best fitness = 20
Generation 100: Best fitness = 20
Generation 150: Best fitness = 20

Optimized Shelf Allocation:
Milk -> Refrigerator Zone
Rice Bag -> Lower Shelf
Frozen Nuggets -> Refrigerator Zone
Cereal -> Eye-Level Shelf
Pasta -> General Aisle Shelf
Pasta Sauce -> General Aisle Shelf
Detergent -> Hazardous Item Zone
Glass Cleaner -> Hazardous Item Zone

Final Fitness Score: 20

Shelf Weight Usage:
Checkout Display: 0.0kg / 8kg
Lower Shelf: 10.0kg / 25kg
Eye-Level Shelf: 3.0kg / 15kg
General Aisle Shelf: 5.0kg / 20kg
Refrigerator Zone: 10.0kg / 20kg
Hazardous Item Zone: 9.0kg / 10kg
Results saved to shelf_optimization_results.xlsx
